In [1]:
import pySPM
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from ipywidgets import interact
from mpl_toolkits.axes_grid1 import make_axes_locatable, Grid, ImageGrid
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True

In [2]:
def plot_pfm(i=0):
    f = pySPM.Bruker(filelist[i])
    va = f.get_channel(channel='Amplitude1',backward=True)
    la = f.get_channel(channel='Amplitude2',backward=True)
    vp = f.get_channel(channel='Phase1',backward=True)
    lp = f.get_channel(channel='Phase2',backward=True)
    ls = [va, la, vp, lp]
    titles = ['Vertical Amplitude','Lateral Amplitude', 'Vertical Phase', 'Lateral Phase']
    
    drivenum = float(f.scanners[0][b'@2:CantDriveContactMode'][0].decode("utf-8")[21:29])
    driveunit = f.scanners[0][b'@2:CantDriveContactMode'][0].decode("utf-8")[30:]
    
    freqnum = float(f.scanners[0][b'@2:CantFrequencyContactMode'][0].decode("utf-8")[25:33])
    frequnit = f.scanners[0][b'@2:CantFrequencyContactMode'][0].decode("utf-8")[34:]

    phasenum = float(f.scanners[0][b'@2:CantPhaseContactMode'][0].decode("latin1")[28:30])
    phaseunit = f.scanners[0][b'@2:CantPhaseContactMode'][0].decode("latin1")[30:]

    deflnum = float(f.scanners[0][b'@2:AFMSetDeflection'][0].decode("utf-8")[40:49])
    deflunit = f.scanners[0][b'@2:AFMSetDeflection'][0].decode("utf-8")[49:]
    
    fdict = dict(verticalalignment='bottom', size='x-large')
    
    xsize = la.size['real']['x']
    ysize = la.size['real']['y']
    
    fig = plt.figure(figsize=(7,7),dpi=100)
    grid = ImageGrid(fig, 111,
                nrows_ncols=(2,2),
                axes_pad = 0.1,
                label_mode='L')
    
    fontprops = fm.FontProperties(size=11)
    scalebarlabel = '{0} {1}'.format(xsize/5, la.size['real']['unit']) 
    scalebar = AnchoredSizeBar(grid[0].transData, xsize/5, scalebarlabel, 'lower right', 
                               color='white', frameon=False, size_vertical=xsize/500, pad=0.5,
                               sep=5, fontproperties=fontprops)
    
    grid[0].add_artist(scalebar)
    grid[0].set_ylabel('Amplitude', fdict)
    grid[0].set_title('Vertical', fdict)
    grid[1].set_title('Lateral', fdict)
    grid[2].set_ylabel('Phase', fdict)
    
    for axis, img, title in zip(grid, ls, titles):
        
        # plot the image
        imgplot = np.flipud(img.pixels)
        im = axis.imshow(imgplot,cmap='afmhot',vmax=imgplot.max()+imgplot.mean()/2, 
                         interpolation='bilinear', extent=[0, xsize, 0, ysize])
        
        # get rid of numbers
        plt.setp(axis.get_xticklabels(), visible=False)
        plt.setp(axis.get_yticklabels(), visible=False)
        # get rid of ticks
        axis.tick_params(axis='both', which='both',length=0)

#         axis.axis('off')
        
#         axis.set_title(title, fdict)
        
#         divider = make_axes_locatable(axis)
#         cax = divider.append_axes('right', size='5%', pad=0.05)
#         cbar = plt.colorbar(im, cax=cax)
#         cbar.set_label(img.zscale, labelpad=0.1)
        
#         axis.set_xlabel('x [nm]')
#         axis.set_ylabel('y [nm]')
        
#         ymin, ymax, = axis.get_ylim()
#         xmin, xmax = axis.get_xlim()
#         axis.set_yticks(np.round(np.linspace(ymin, ymax, 2), 2))
#         axis.set_xticks(np.round(np.linspace(xmin, xmax, 2), 2))

    plt.gcf().text(0.5, 0.08,r'\textbf{{Setpoint}}={0} {1} | \textbf{{Drive}}={2} {3} | \textbf{{Freq.}}={4} {5} | \textbf{{Phase}}={6} {7}'.format(
                                                                                              deflnum, deflunit,
                                                                                              drivenum, driveunit,
                                                                                              freqnum, frequnit, 
                                                                                              phasenum, phaseunit), 
                   size=10, weight='normal', va='bottom', ha='center', bbox=dict(boxstyle="round", facecolor='orange', alpha=0.1))
#     plt.tight_layout()
    plt.show()

In [3]:
afmdir = '/users/zatterin/Documents/data/AFM/kto_pto/'
%ls {afmdir}

e16014/  e16019/  e16050/  e16065/  e16089/  e17049/  e17089/  e18015/
e16018/  e16020/  e16056/  e16088/  e16095/  e17087/  e17090/


In [4]:
sample = 'e16020/'
datefolders = !ls {afmdir+sample}
for index, date in enumerate(datefolders): print('{0}...{1}/'.format(index, date)) 

0...03.05/
1...06.05/
2...07.09/
3...09.05/
4...11.11/
5...NsThumbnails.bin/


In [5]:
i = 1
filedir = afmdir+sample+datefolders[i]+'/'
%ls {filedir}

05061922.001  e16020E.004  e16020E.008  e16020E.011  png/
e16020E.001   e16020E.006  e16020E.009  e16020E.012
e16020E.002   e16020E.007  e16020E.010  e16020E.013


In [47]:
filelist = !ls {filedir} | grep '0'
for index in range(len(filelist)): 
    filelist[index] = filedir+filelist[index]

In [27]:
interact(plot_pfm, i=(2, len(filelist)-1, 1));

interactive(children=(IntSlider(value=2, description='i', max=24, min=2), Output()), _dom_classes=('widget-interact',))

## The old way

In [6]:
sample = 'e16020/'
datefolders = !ls {afmdir+sample}
for index, date in enumerate(datefolders): print('{0}...{1}/'.format(index, date)) 

0...03.05/
1...06.05/
2...07.09/
3...09.05/
4...11.11/
5...NsThumbnails.bin/


In [7]:
i = 3
filedir = afmdir+sample+datefolders[i]+'/'
%ls {filedir}

e16020E.014  e16020E.019  e16020E.024  e16020E.029       pfm_09.05.16.pptx
e16020E.015  e16020E.020  e16020E.025  e16020E.030       png/
e16020E.016  e16020E.021  e16020E.026  e16020E.031
e16020E.017  e16020E.022  e16020E.027  e16020E.032
e16020E.018  e16020E.023  e16020E.028  pfm_09.05.16.pdf


In [8]:
filelist = !ls {filedir} | grep '0'
for index in range(len(filelist)): 
    filelist[index] = filedir+filelist[index]

In [24]:
@interact(i=(0, len(filelist)-1, 1))
def plot_pfm_old(i=0):
    f = pySPM.Bruker(filelist[i])
    a = f.get_channel(channel='Amplitude1',backward=True)
    p = f.get_channel(channel='Phase1',backward=True)
    ls = [a, p]
    
    xsize = a.size['real']['x']
    ysize = a.size['real']['y']
    
    fdict = dict(verticalalignment='bottom', size='x-large')
        
    fig = plt.figure(figsize=(7,7),dpi=100)
    grid = ImageGrid(fig, 111,
                nrows_ncols=(1,2),
                axes_pad = 0.1,
                label_mode='L')
    
    fontprops = fm.FontProperties(size=11)
    scalebarlabel = '{0} {1}'.format(np.round(xsize/5, decimals=1), a.size['real']['unit']) 
    scalebar = AnchoredSizeBar(grid[0].transData, xsize/5, scalebarlabel, 'lower right', 
                               color='white', frameon=False, size_vertical=xsize/500, pad=0.5,
                               sep=5, fontproperties=fontprops)
    
    grid[0].add_artist(scalebar)
    grid[0].set_title('Amplitude', fdict)
    grid[1].set_title('Phase', fdict)
    
    for axis, img in zip(grid, ls):
    
        # plot the image
        imgplot = np.flipud(img.pixels)
        im = axis.imshow(imgplot,cmap='afmhot',vmax=imgplot.max()+imgplot.mean()/2, 
                         interpolation='bilinear', extent=[0, xsize, 0, ysize])

        # get rid of numbers
        plt.setp(axis.get_xticklabels(), visible=False)
        plt.setp(axis.get_yticklabels(), visible=False)
        # get rid of ticks
        axis.tick_params(axis='both', which='both',length=0)

interactive(children=(IntSlider(value=0, description='i', max=20), Output()), _dom_classes=('widget-interact',))